# Movies Recommendation systems

In [2]:
import os
cwd = os.getcwd()
print(cwd)

chdir=os.chdir("D:\Folder D/New folder/Projects done/Movies Recommendation/archive/")
print(chdir)

D:\Folder D\New folder\Projects done\Movies Recommendation\archive
None


# Import Libraries and Data

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD 
from surprise.model_selection import cross_validate

import warnings; warnings.simplefilter('ignore')

In [8]:
data= pd. read_csv('movies_metadata.csv',header=0)
data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


## Explorartry data Analysis

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [10]:
data.describe()

,revenue,runtime,vote_average,vote_count
count,4.546000e+04,45203.000000,45460.000000,45460.000000
mean,1.120935e+07,94.128199,5.618207,109.897338
std,6.433225e+07,38.407810,1.924216,491.310374
min,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,107.000000,6.800000,34.000000
max,2.787965e+09,1256.000000,10.000000,14075.000000


In [11]:
data.describe

<bound method NDFrame.describe of        adult                              belongs_to_collection    budget  \
0      False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1      False                                                NaN  65000000   
2      False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3      False                                                NaN  16000000   
4      False  {'id': 96871, 'name': 'Father of the Bride Col...         0   
...      ...                                                ...       ...   
45461  False                                                NaN         0   
45462  False                                                NaN         0   
45463  False                                                NaN         0   
45464  False                                                NaN         0   
45465  False                                                NaN         0   

                                         

## Content- Based Filtering alogrithm

### Build the algorithm based on Movie Overviews and Taglines

In [12]:
CBF = pd.read_csv('links_small.csv')
print(CBF.head())

   movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0


In [13]:
CBF = CBF[CBF['tmdbId'].notnull()]['tmdbId'].astype('int')
data = data.drop([19730, 29503, 35587])

In [14]:
# Check the number of the data points that the smaller verision are in the biggest one
data['id'] = data['id'].astype('int')
data2 = data[data['id'].isin(CBF)]
data2.shape

(9099, 24)

In [17]:
data2['tagline'] = data2['tagline'].fillna('')
data2['description'] = data2['overview'] + data2['tagline']
data2['description'] = data2['description'].fillna('')

In [18]:
tfidf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tf_matrix = tfidf.fit_transform(data2['description'])
tf_matrix.shape

(9099, 268124)

In [20]:
# I will aplly the cosine similarity to find out the similarities between the movies descriptions . 
#Therefore, we will use sklearn's linear_kernel instead of cosine_similarities since it is much faster.

cosine_sim = linear_kernel(tf_matrix, tf_matrix)
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

In [21]:
# Reset the index to make the titles as an index so we can see the recommendations of a specific movie
data2 = data2.reset_index()
titles = data2['title']
print(titles.head())
indices = pd.Series(data2.index, index=data2['title'])
print(indices.head())

0                      Toy Story
1                        Jumanji
2               Grumpier Old Men
3              Waiting to Exhale
4    Father of the Bride Part II
Name: title, dtype: object
title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [22]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [23]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [24]:
get_recommendations('Batman Forever').head(10)

7931                      The Dark Knight Rises
2579               Batman: Mask of the Phantasm
6900                            The Dark Knight
6144                              Batman Begins
8165    Batman: The Dark Knight Returns, Part 1
524                                      Batman
1240                             Batman & Robin
1113                             Batman Returns
7565                 Batman: Under the Red Hood
7901                           Batman: Year One
Name: title, dtype: object

### Build the algorithm based on genres, director, main actors and keywords.

In [25]:
credits = pd.read_csv('credits.csv')
keywords = pd.read_csv('keywords.csv')

In [26]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
data['id'] = data['id'].astype('int')
data.shape

(45463, 24)

In [27]:
data = data.merge(credits, on='id')
data = data.merge(keywords, on='id')

In [28]:
data2 = data[data['id'].isin(CBF)]
data2.shape

(9219, 27)

In [29]:
#Make sure that the data types in the all features are abled to  be expressed on python 
data2['cast'] = data2['cast'].apply(literal_eval)
data2['crew'] = data2['crew'].apply(literal_eval)
data2['keywords'] = data2['keywords'].apply(literal_eval)
data2['cast_size'] = data2['cast'].apply(lambda x: len(x))
data2['crew_size'] = data2['crew'].apply(lambda x: len(x))

In [30]:
def director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [31]:
data2['director'] = data2['crew'].apply(director)

In [32]:
data2['cast'] = data2['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
data2['cast'] = data2['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [33]:
data2['keywords'] = data2['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [34]:
# i will clean and improve the text data , so we can get a good recommendations
data2['cast'] = data2['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [35]:
data2['director'] = data2['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
# Mention Director 3 times to give it more weight relative to the entire cast
data2['director'] = data2['director'].apply(lambda x: [x,x, x])

In [36]:
# we calculate the frequenct counts of every keyword that appears in the dataset.
count = data2.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
count.name = 'keyword'
count = count.value_counts()
count[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

In [37]:
# We will apply steeming , sowe can get the root of the words in the keywords
count = count[count > 1]
stemmer = SnowballStemmer('english')
stemmer.stem('Classes')

'class'

In [38]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in count:
            words.append(i)
    return words

In [39]:
data2['keywords'] = data2['keywords'].apply(filter_keywords)
data2['keywords'] = data2['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
data2['keywords'] = data2['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [40]:
print(data2['genres'])

0        [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1        [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2        [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3        [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                           [{'id': 35, 'name': 'Comedy'}]
                               ...                        
40952                        [{'id': 18, 'name': 'Drama'}]
41172    [{'id': 53, 'name': 'Thriller'}, {'id': 10749,...
41225    [{'id': 12, 'name': 'Adventure'}, {'id': 18, '...
41391    [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
41669    [{'id': 99, 'name': 'Documentary'}, {'id': 104...
Name: genres, Length: 9219, dtype: object


In [42]:

data2['CBF2'] = data2['keywords'] + data2['cast'] + data2['director'] 
data2['CBF2'] = data2['CBF2'].apply(lambda x: ' '.join(x))

In [43]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(data2['CBF2'])

In [44]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [45]:
data2 = data2.reset_index()
titles = data2['title']
indices = pd.Series(data2.index, index=data2['title'])

In [46]:
get_recommendations('The Dark Knight').head(10)

8031         The Dark Knight Rises
6218                 Batman Begins
6623                  The Prestige
7648                     Inception
2085                     Following
4145                      Insomnia
3381                       Memento
8613                  Interstellar
7659    Batman: Under the Red Hood
1134                Batman Returns
Name: title, dtype: object

## Collaboritive Filtering using Singular Value Decomposition (SVD)

In [47]:
reader = Reader()

In [48]:
ratings = pd.read_csv('ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [54]:
data3 = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [52]:
svd = SVD()
cross_validate(svd, data3, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8955  0.8970  0.8981  0.8981  0.8920  0.8961  0.0023  
MAE (testset)     0.6861  0.6920  0.6933  0.6929  0.6866  0.6902  0.0031  
Fit time          6.22    5.69    5.71    5.72    6.07    5.88    0.22    
Test time         0.14    0.15    0.15    0.16    0.25    0.17    0.04    


{'test_rmse': array([0.8954673 , 0.89704132, 0.89808348, 0.8981458 , 0.89200872]),
 'test_mae': array([0.68612754, 0.69202526, 0.69327975, 0.69290988, 0.68662877]),
 'fit_time': (6.217609643936157,
  5.6860573291778564,
  5.7059431076049805,
  5.719002962112427,
  6.065642833709717),
 'test_time': (0.14373159408569336,
  0.15163612365722656,
  0.14803528785705566,
  0.15828967094421387,
  0.24592900276184082)}

In [57]:
trainset = data3.build_full_trainset()
svd.fit(trainset)

In [58]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [59]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.4933536228001936, details={'was_impossible': False})

## Hybird filtering 

In [60]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [61]:
id_map = pd.read_csv('links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(data2[['title', 'id']], on='id').set_index('title')

In [62]:
indices_map = id_map.set_index('id')

In [67]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = data2.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [72]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,id,est
987,Alien,4564.0,7.9,348,3.360737
974,Aliens,3282.0,7.7,679,3.181434
1011,The Terminator,4208.0,7.4,218,3.094174
8401,Star Trek Into Darkness,4479.0,7.4,54138,3.016842
899,Platoon,1236.0,7.5,792,2.980720
4987,Battle Royale,992.0,7.3,3176,2.943018
907,A Christmas Carol,34.0,7.0,25842,2.862356
522,Terminator 2: Judgment Day,4274.0,7.7,280,2.861893
2014,Fantastic Planet,140.0,7.6,16306,2.818935
4260,The Four Feathers,164.0,6.5,9093,2.725142


In [69]:
hybrid(500, 'Avatar')

,title,vote_count,vote_average,id,est
899,Platoon,1236.0,7.5,792,3.669209
8107,Wrath of the Titans,1459.0,5.5,57165,3.275318
4260,The Four Feathers,164.0,6.5,9093,3.266990
987,Alien,4564.0,7.9,348,3.225457
4987,Battle Royale,992.0,7.3,3176,3.207749
1376,Titanic,7770.0,7.5,597,3.097572
8231,Side by Side,111.0,7.1,110354,3.046580
7065,Meet Dave,381.0,5.1,11260,3.038589
8934,Home,1539.0,6.8,228161,2.990179
8096,John Carter,2170.0,6.1,49529,2.958751
